In [0]:
#Librerias que se usaran
import pandas as pd  # Libreria para administrar tablas, y realizar trabajos con distintas formas de tablas o dataframes
import numpy as np   # Libreria para poder hacer operaciones matematicas y matriciales
import matplotlib.pyplot as plt # Libreria para realizar graficos 
from tabulate import tabulate   # Permite formatear y mostrar de mejor manera los datos tabulares
import seaborn as sns  # Libreria para realizar graficos y vizualizaciones
import psycopg2        # Libreria que permite la conexion con PostgresSQL
from matplotlib.backends.backend_pdf import PdfPages # Libreria que permite exportar graficos en pdf

#Configuramos pandas para que podamos vizualizar todas las columnas y filas la estadistica descriptiva de todas las variables
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
#Configuramos pandas para que lanze valores con una precision de hasta 6 decimales
pd.set_option('display.float_format', '{:.6f}'.format)

In [0]:
# Información de la conexión a PostgreSQL
host = "srv-postgres-d4m.postgres.database.azure.com"
database = "ControlSenseDB"
user = "administrador"
password = "Protobuffers2024"

# Establecer la conexión a la base de datos
try:
    connection = psycopg2.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    # Crear un cursor para ejecutar comandos SQL
    cursor = connection.cursor()

    # Establecer la zona horaria antes de ejecutar la consulta
    time_zone_query = "SET TIME ZONE 'America/Lima';"
    cursor.execute(time_zone_query)

    # Tu consulta SQL equipos de acarreo a considerar en c4mequipos y datacamion: 61,63,64,65,68,69,71,74,103,122,125,126,198,199,201,204,205
    # equipos acarreo disponibles en telemetria h4m(2dias) : 61,65,68,126,199,201,*204*,205
    tu_query_sql = '''
    SELECT *
    FROM public.getsensorsvaluesmod(205, '2024-02-27 07:00:30.612-05', '2024-03-03 06:27:30.612-05') a --data_camion
    left join tp_tramosstat b
    on b.id = (select id from tp_tramosstat where id_tptramosstat = a.tramosidsnew_t and tiem_creac < a.instant_date_t
                    order by tiem_creac desc limit 1)
    where b.nombre_tramo is not null
    order by a.instant_date_t asc
    '''
    # Ejecutar la consulta
    cursor.execute(tu_query_sql)

    # Obtener los resultados en un DataFrame de pandas
    resultados14 = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in cursor.description])

    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

    # Hacer lo que necesites con los resultados
    print(resultados14.head())

except psycopg2.Error as e:
    print("Error al conectar a la base de datos PostgreSQL:", e)

#### Metodo 2: Guardarmos directamente el dfpandas en un csv en el BlobStorage

In [0]:
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

# Convertir el DataFrame de Pandas a CSV
resultados14.to_csv("datos_raw_shougang_equipo205.csv", index=False)

# # Obtener la conection, del Azure DataLake, se encuentra en la interfaz de Azure (Claves de acceso: Key1)
# conection_string = 'DefaultEndpointsProtocol=https;AccountName=datalakemlopsd4m;AccountKey=iWT8t74/#XlqcqoR03keDVtFZPzr0PB9zDffMPaLWMUBIAjUww8uYAVkc9xRkcBtvTmUHKBvd1sB3+ASt6mGgcQ==;EndpointSuffix=core.windows.net'

# Conectar al Blob Storage de Azure
blob_service_client = BlobServiceClient.from_connection_string(conection_string)

# Nombre del contenedor y archivo en el Blob Storage
container_name = "raw/proyectocongestion_raw/fuentedatos_c4m/operacion_shougang/"
blob_name = "datos_raw_shougang_equipo205.csv"

# Subir el archivo CSV al Blob Storage
with open("datos_raw_shougang_equipo205.csv", "rb") as data:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data)